In [135]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
import py4vasp as pv

In [136]:
data_mace=pd.read_pickle('/Users/dominicwelti/Documents/Master_Thesis_Data_Set/Graphite/MACE/results.pkl')

In [137]:
dft_folder='/Users/dominicwelti/Documents/Master_Thesis_Data_Set/Graphite/DFT'
files=list(filter(lambda x:'.h5' in x, os.listdir(dft_folder)))

data_dft=pd.DataFrame({
    'Calculation type': 'DFT',
    'Model': '',
    'c Factor': [0]*len(files),
    'c Parameter': 0,
    'Energy': 0
})

In [138]:
for i, row in data_dft.iterrows():
    data_dft.loc[i, 'Model'] = files[i].split('_')[0]
    data_dft.loc[i, 'c Factor'] = files[i].split('_')[1].split('.h')[0]

    calc=pv.Calculation.from_file(f'{dft_folder}/{files[i]}')
    atoms=calc.structure.to_ase()
    data_dft.loc[i, 'c Parameter'] = atoms.get_cell()[2,2]
    data_dft.loc[i, 'Energy'] = calc.energy.read()['energy(sigma->0)']


In [139]:
def normalize(df:pd.DataFrame):
    '''Normalize energy values (delta to lowest value of model)'''
    df['Relative energy']=0
    for model in df['Model'].unique():
        min_e, max_e = np.min(df.loc[df['Model']==model, 'Energy']), np.max(df.loc[df['Model']==model, 'Energy'])
        df.loc[df['Model']==model, 'Relative energy'] = (-df.loc[df['Model']==model, 'Energy'] + min_e) / (-max_e+min_e)
    return df

In [140]:
data_dft=normalize(data_dft)
data_mace=normalize(data_mace)

In [141]:
df=pd.concat([data_mace, data_dft])
df['c Factor']=df['c Factor'].astype(float)
df=df.sort_values('c Factor').reset_index(drop=True)

In [142]:
plot=px.line(df, x='c Factor', y='Relative energy', color='Model', markers=True, template='ggplot2')

In [143]:
plot.show()